In [2]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import librosa as lb
import librosa.display
from sklearn.model_selection import train_test_split
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras import layers,models,Sequential
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory
import tensorflow as tf
import os
import gc
from PIL import Image as PIL_Image
# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

**For the audio classification**
<br>
1) We take audio create 3 channeled image data for the audio
2) Just like how we take rgb in the image we need 3 channels for the audio it would be<br>
       i) Fourier transformation (more of frequency over time)stft<br>
       ii) meft (image displaying frequencey that we humnas can listen to)<br>
       iii)chroma(smaller pitches and notes)

In [ ]:

def format_shape(data, target_height=128, target_width=1000):
    data = np.asarray(data, dtype=np.float32)

    
    # 1D -> 2D
    if data.ndim == 1:
        data = data[np.newaxis, :]

    # Min-max normalization
    data=(255 * (data - np.min(data)) / (np.max(data) - np.min(data))).astype(np.uint8)

    # Resize rows to target_height
    rows, cols = data.shape
    if rows < target_height:
        reps = int(np.ceil(target_height / rows))
        data = np.tile(data, (reps, 1))[:target_height, :]
    elif rows > target_height:
        data = data[:target_height, :]

    # Resize columns to target_width
    rows, cols = data.shape
    if cols < target_width:
        pad_width = target_width - cols
        data = np.pad(data, ((0,0), (0, pad_width)), mode="constant")
    elif cols > target_width:
        data = data[:, :target_width]

    return data.astype(np.float32)


In [ ]:
#this function will be responsible to convert all the audios into images
def audio_to_image(file=None,max_size=1000,y=None,sr=22050):
    #loading up the image
    if not file is None:
        if y is None:
            y,sr=librosa.load(file,sr=22050)
    y = np.asarray(y, dtype=np.float32)

    mels = librosa.feature.melspectrogram(y=y, sr=sr, n_mels=128, n_fft=2048, hop_length=512)

    #next channel which mfcc is which is audio graph that is audible to human
    mels_db = librosa.power_to_db(mels, ref=np.max)
    

    
    mels_delta = librosa.feature.delta(mels_db)
    
    mels_delta2 = librosa.feature.delta(mels_db, order=2)

    def normalize(X):
        x_min, x_max = X.min(), X.max()
        if x_max - x_min > 0:
            return (255 * (X - x_min) / (x_max - x_min)).astype(np.uint8)
        return np.zeros_like(X, dtype=np.uint8)
        
    layer0 = format_shape(mels_db)     
    layer1 = format_shape(mels_delta)  
    layer2 = format_shape(mels_delta2)

    #this makes cube by taking 3 images and stacking on top of each other
    final_image = np.dstack([layer0, layer1, layer2]).astype(np.float32)
    return final_image,y,sr
    

In [ ]:
def convert_dir_to_audio(file_path, output_dir):
    class_names = [d for d in os.listdir(file_path) if os.path.isdir(os.path.join(file_path, d))]

    for i, class_name in enumerate(class_names):
        class_input_path = os.path.join(file_path, class_name)
        save_folder = os.path.join(output_dir, class_name)
        os.makedirs(save_folder, exist_ok=True)
        
        print(f"--- Processing Class: {class_name} ---")
        
        for file_name in os.listdir(class_input_path):
            if file_name.endswith((".wav", ".m4a")):
                full_file_path = os.path.join(class_input_path, file_name)

                try:
                    # 1. Process Original
                    # Note: We use 'pixel_data' everywhere now
                    pixel_data, y, sr = audio_to_image(full_file_path, y=None)
                    
                    # Save Original
                    im = PIL_Image.fromarray(pixel_data.astype(np.uint8))
                    im.save(os.path.join(save_folder, f"{file_name}_orig.png"))
                    
                    # 2. Process Augmentations
                    for j in range(3): 
                        y_aug = augment_audio(y, sr)
                        aug_pixel_data, _, _ = audio_to_image(file=None, y=y_aug, sr=sr)
                        
                        im_aug = PIL_Image.fromarray(aug_pixel_data.astype(np.uint8))
                        # Use f-string to safely combine string and number j
                        im_aug.save(os.path.join(save_folder, f"{file_name}_aug_{j}.png"))
                        
                        del y_aug, aug_pixel_data 

                    # Cleanup the original data for this file
                    del pixel_data, y
                    
                except Exception as e:
                    print(f"Skipping {file_name}: {e}")
        
        gc.collect()

    return class_names

In [ ]:
def augment_audio(y,sr):
    if np.random.random()>0.5:
        y=librosa.effects.pitch_shift(y,sr=sr,n_steps=np.random.uniform(-2,2))
        y = np.asarray(y, dtype=np.float32)
    noise_amp=0.005*np.random.uniform()*np.amax(y)
    noise_amp = 0.005 * np.random.uniform() * np.amax(np.abs(y)) if y.size > 0 else 0.0
    y = y + noise_amp * np.random.normal(size=y.shape).astype(np.float32)
    return y.astype(np.float32)

In [ ]:
def for_single_audio(file):
    img,y,sr=audio_to_image(file)
    img_array=np.expand_dims(img_array, axis=0)

In [ ]:
input="/kaggle/input/forest-sound-dataset/forestdataset"
output="/kaggle/working/processed"
class_names=convert_dir_to_audio(input,output)



In [ ]:
import shutil

# Zip the folder for easy download
shutil.make_archive("/kaggle/working/processed", 'zip', "/kaggle/working/forest_processed")

In [3]:
from tensorflow.keras.utils import image_dataset_from_directory, load_img, img_to_array
os.environ["TF_CPP_MIN_LOG_LEVEL"]="3"

In [4]:
#setting up important var for loading images later on
BATCH_SIZE=32
IMG_SIZE=(224,224)
SEED=42
EXTRACT_PATH="/kaggle/input/forest-sound-spectograph/forest_processed"
selected_class=["natural sound","unnatural"]

In [5]:
#loading up images and dividing them for the neural net
train_data=tf.keras.utils.image_dataset_from_directory(
    EXTRACT_PATH,
    image_size=(128, 1000),
    class_names=selected_class,
    validation_split=0.2,
    subset="training",
    seed=SEED,
    batch_size=BATCH_SIZE,
    label_mode="binary",
    color_mode='rgb'
)

#validation dataset
validation_data=tf.keras.utils.image_dataset_from_directory(
    EXTRACT_PATH,
    image_size=(128, 1000),
    class_names=selected_class,
    validation_split=0.2,
    subset="validation",
    seed=SEED,
    batch_size=BATCH_SIZE,
    label_mode="binary",
    color_mode='rgb'
)
#now dividing the validation data set into 
total_number_of_batches_in_validation_data=validation_data.cardinality().numpy() #converts total number of batches in the set and converts the tensor into python integer with .numpy function here
no_of_batches_in_validation_data=total_number_of_batches_in_validation_data//2 #this is a floor division operator


#now from the set of batches of the image creating subset into the validation and the test subset
validation=validation_data.take(no_of_batches_in_validation_data)
test=validation_data.skip(no_of_batches_in_validation_data)
print(f"Class names: {train_data.class_names}")
print("For values:\n")
for i, class_name in enumerate(train_data.class_names):
    print(f"{class_name}:{i}\n")

Found 8016 files belonging to 2 classes.
Using 6413 files for training.


I0000 00:00:1767503196.317802      55 gpu_device.cc:2019] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 13942 MB memory:  -> device: 0, name: Tesla T4, pci bus id: 0000:00:04.0, compute capability: 7.5
I0000 00:00:1767503196.321715      55 gpu_device.cc:2019] Created device /job:localhost/replica:0/task:0/device:GPU:1 with 13942 MB memory:  -> device: 1, name: Tesla T4, pci bus id: 0000:00:05.0, compute capability: 7.5


Found 8016 files belonging to 2 classes.
Using 1603 files for validation.
Class names: ['natural sound', 'unnatural']
For values:

natural sound:0

unnatural:1



In [6]:
#now we take pretrained model and train them for our data

data_augmentation=tf.keras.Sequential([
     tf.keras.layers.RandomTranslation(
        height_factor=0.0,  # no vertical shift by default
        width_factor=0.1,   # shift up to 10% horizontally
        fill_mode='constant'
    ),

    # Random vertical shift (frequency axis)
    tf.keras.layers.RandomTranslation(
        height_factor=0.1,  # shift up to 10% vertically
        width_factor=0.0,
        fill_mode='constant'
    ),

    # Random brightness adjustment (small)
    tf.keras.layers.RandomBrightness(factor=0.1),

    # Random contrast adjustment
    tf.keras.layers.RandomContrast(factor=0.1),



])



In [7]:
def binary_model():
    # 1. Define Input
    inputs = tf.keras.Input(shape=(128, 1000, 3))
    x = data_augmentation(inputs)
    x = layers.Resizing(224, 224)(x)


    base_model = tf.keras.applications.EfficientNetB0(
        input_shape=(224, 224, 3),
        include_top=False,
        weights="imagenet"
    )
    base_model.trainable = False
    
    x = base_model(x, training=False) # training=False keeps BatchNormalization in inference mode

    # 5. The "Head"
    x = layers.GlobalMaxPooling2D()(x)
    x = layers.Dense(256, activation="relu")(x)
    outputs = layers.Dense(1, activation="sigmoid")(x)

    # 6. Create the Model
    model = models.Model(inputs=inputs, outputs=outputs)
    
    return model
model = binary_model()

model.compile(optimizer="adam",
             loss="binary_crossentropy",
             metrics=["accuracy"])

history=model.fit(
    train_data,
    validation_data=validation,
    epochs=50,
    batch_size=32,
    callbacks=[
        EarlyStopping(
            monitor="val_accuracy", #checks for val_accuracy
            patience=5,#wait tills 5 epochs
            restore_best_weights=True,#uses best weight
        ),
        ModelCheckpoint(
            "best_model.keras",#givesbest model according to val_accuracy
            monitor="val_accuracy",
            save_best_only=True,
            verbose=1 #prints only certain line of epoch for 1 and for 0 is silence and for 2 is every line
        )
    ]
)
test_loss,test_acc=model.evaluate(validation)
print(f"Accuracy for validation dataset: {test_acc:4f}")

16705208/16705208 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step
Epoch 1/50


E0000 00:00:1767503211.847212      55 meta_optimizer.cc:967] layout failed: INVALID_ARGUMENT: Size of values 0 does not match size of permutation 4 @ fanin shape inStatefulPartitionedCall/functional_1_1/efficientnetb0_1/block2b_drop_1/stateless_dropout/SelectV2-2-TransposeNHWCToNCHW-LayoutOptimizer
I0000 00:00:1767503214.141865     136 cuda_dnn.cc:529] Loaded cuDNN version 91002


201/201 ━━━━━━━━━━━━━━━━━━━━ 0s 74ms/step - accuracy: 0.7264 - loss: 0.9143
Epoch 1: val_accuracy improved from -inf to 0.77125, saving model to best_model.keras
201/201 ━━━━━━━━━━━━━━━━━━━━ 34s 104ms/step - accuracy: 0.7266 - loss: 0.9125 - val_accuracy: 0.7713 - val_loss: 0.5595
Epoch 2/50
200/201 ━━━━━━━━━━━━━━━━━━━━ 0s 76ms/step - accuracy: 0.7998 - loss: 0.4208
Epoch 2: val_accuracy improved from 0.77125 to 0.84250, saving model to best_model.keras
201/201 ━━━━━━━━━━━━━━━━━━━━ 18s 87ms/step - accuracy: 0.7999 - loss: 0.4205 - val_accuracy: 0.8425 - val_loss: 0.3373
Epoch 3/50
200/201 ━━━━━━━━━━━━━━━━━━━━ 0s 76ms/step - accuracy: 0.8333 - loss: 0.3759
Epoch 3: val_accuracy improved from 0.84250 to 0.86125, saving model to best_model.keras
201/201 ━━━━━━━━━━━━━━━━━━━━ 18s 88ms/step - accuracy: 0.8334 - loss: 0.3758 - val_accuracy: 0.8612 - val_loss: 0.3071
Epoch 4/50
200/201 ━━━━━━━━━━━━━━━━━━━━ 0s 77ms/step - accuracy: 0.8350 - loss: 0.3765
Epoch 4: val_accuracy improved from 0.861

In [15]:
# ===== ADD THIS CELL AFTER model.fit() COMPLETES =====

print("\n" + "="*80)
print("🔍 VERIFYING MODEL BEFORE SAVING")
print("="*80)

# Check the model we just trained
print(f"Model input shape: {model.input_shape}")
print(f"Expected:          (None, 128, 1000, 3)")

# Test with a sample batch
for images, labels in train_data.take(1):
    print(f"\nSample batch shape: {images.shape}")
    print(f"Sample batch channels: {images.shape[-1]}")
    
    # Try prediction
    try:
        pred = model.predict(images[:1])
        print(f"✅ Model can predict on training data")
        print(f"   Prediction shape: {pred.shape}")
    except Exception as e:
        print(f"❌ Model CANNOT predict: {e}")
        print("   TRAINING DATA AND MODEL DON'T MATCH!")

if model.input_shape == (None, 128, 1000, 3):
    print("\n✅ MODEL ARCHITECTURE IS CORRECT")
else:
    print(f"\n❌ MODEL ARCHITECTURE IS WRONG!")
    print(f"   Model expects: {model.input_shape}")
    print(f"   But should be: (None, 128, 1000, 3)")
    raise ValueError("Model has wrong input shape! Fix training code!")

print("="*80 + "\n")

# Now save
os.makedirs("/kaggle/working/models", exist_ok=True)
model.save("/kaggle/working/models/audio_forest_69.keras")

# Verify the SAVED file
print("\n" + "="*80)
print("🔍 VERIFYING SAVED FILE")
print("="*80)

test_model = tf.keras.models.load_model(
    "/kaggle/working/models/audio_forest_69.keras", 
    compile=False
)

print(f"Saved model input shape: {test_model.input_shape}")

if test_model.input_shape == (None, 128, 1000, 3):
    print("✅ SAVED MODEL IS CORRECT - Safe to download!")
else:
    print("❌ SAVED MODEL IS WRONG!")
    raise ValueError("Something went wrong during save!")
    
print("="*80 + "\n")

# Zip it
import shutil
shutil.make_archive("/kaggle/working/models", 'zip', "/kaggle/working/models")
print("✅ models.zip created - MODEL IS VERIFIED CORRECT!")


🔍 VERIFYING MODEL BEFORE SAVING
Model input shape: (None, 128, 1000, 3)
Expected:          (None, 128, 1000, 3)

Sample batch shape: (32, 128, 1000, 3)
Sample batch channels: 3
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 41ms/step
✅ Model can predict on training data
   Prediction shape: (1, 1)

✅ MODEL ARCHITECTURE IS CORRECT


🔍 VERIFYING SAVED FILE
Saved model input shape: (None, 128, 1000, 3)
✅ SAVED MODEL IS CORRECT - Safe to download!

✅ models.zip created - MODEL IS VERIFIED CORRECT!


In [8]:
result=model.evaluate(test)
print(f"For the unseen test data of the entire training this model has accuracy of {result[1]:.4f}\\")

26/26 ━━━━━━━━━━━━━━━━━━━━ 3s 84ms/step - accuracy: 0.8997 - loss: 0.2479
For the unseen test data of the entire training this model has accuracy of 0.9041\


In [11]:

os.makedirs("/kaggle/working/models", exist_ok=True)
model.save("/kaggle/working/models/audio_forest_3.keras")

# ✅ VERIFY THE SAVED MODEL
print("\n" + "="*60)
print("🔍 VERIFYING SAVED MODEL")
print("="*60)

test_model = tf.keras.models.load_model(
    "/kaggle/working/models/audio_forest_1.keras", 
    compile=False
)

print(f"Model input shape: {test_model.input_shape}")
print(f"Expected:          (None, 128, 1000, 3)")

if test_model.input_shape == (None, 128, 1000, 3):
    print("✅ MODEL IS CORRECT - Safe to download!")
else:
    print("❌ MODEL IS WRONG - DO NOT DOWNLOAD!")
    print("   Something went wrong during training!")
    raise ValueError("Model has wrong input shape!")
    
print("="*60 + "\n")

# Now zip it
import shutil
shutil.make_archive("/kaggle/working/models", 'zip', "/kaggle/working/models")
print("✅ models.zip created and verified!")


🔍 VERIFYING SAVED MODEL
Model input shape: (None, 128, 1000, 3)
Expected:          (None, 128, 1000, 3)
✅ MODEL IS CORRECT - Safe to download!

✅ models.zip created and verified!


**No of files:**
<br>
1) Fire=336<br>
2) Logging=455<br>
3) Natural=1063<br>
4) Poaching=531<br>
5) Total= 2385<br>
So, threshold accuracy=(531/2385)*100%=22.26%